README: <br>
idea: graph-based --> compute cosine similarity for each node (each node represent a user review)

reference: <br>
- https://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelSpreading.html#sklearn.semi_supervised.LabelSpreading <br>
- https://scikit-learn.org/stable/modules/label_propagation.html
- https://scikit-learn.org/stable/auto_examples/semi_supervised/plot_label_propagation_digits.html (usage examples)

In [1]:
# TODO: semi-supervised learning
# build basic semi-supervised learning pipeline

In [1]:
import pandas as pd
import os
os.chdir('/Users/liyuan/desktop/CSAir/codes')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import time
import json

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import confusion_matrix, classification_report

from semi_supervise import Semi_Supervise

In [5]:
ss = Semi_Supervise()
labeled_data = ss.load_labeled_data('../res/all_labeled_data.csv')
# labeled_data = ss.load_labeled_data('../res/labeled_data_with_without_tk.csv')
unlabeled_data = ss.load_unlabeled_data('../res/tokenized_sampled_unlabeled_reviews.txt')
data_concat = ss.concat_data()
data_concat.head()

there are 1700 examples in labeled dataset


,review_tokens,label_encoded
0,优惠券,9.0
1,优惠券,9.0
2,买票 提醒 优惠券,9.0
3,专享 优惠 入口 机票 预订 选择 添加 请问 情况,9.0
4,参加 月份 会员 日 活动 抽 中国 往返机票 免票 客服 答复 说 月 日 发放 优惠券 ...,9.0


In [6]:
start_time = time.time()
ss.get_X()
predicted_labels = ss.label_propagation()
print("--- %s seconds ---" % (time.time() - start_time))

[1700 1701 1702 ... 6462 6463 6464]
y_train: [ 9.  9.  9. ... -1. -1. -1.]
true labels:  [nan nan nan ... nan nan nan]
predicted labels: [9. 1. 5. ... 3. 3. 3.]
--- 613.6748068332672 seconds ---


/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:297: RuntimeWarning: divide by zero encountered in true_divide
  self.label_distributions_ /= normalizer
/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


In [37]:
# match predicted labels to reviews
# unlabeled_data = ss.match_reviews_to_pred_labels('../res/labels_encoded_dict.txt')
# unlabeled_data.head()

In [36]:
# match predicted labels to reviews
unlabeled_data['predicted_labels'] = predicted_labels
with open('../res/labels_encoded_dict.txt','r') as f:
    labels_encoded_dict = json.load(f)
    print(labels_encoded_dict)
    
def get_class(row):
    for k in list(labels_encoded_dict.keys()):
        if labels_encoded_dict[k] == row:
            return k
    
unlabeled_data['predicted_class'] = unlabeled_data['predicted_labels'].apply(get_class)
unlabeled_data.head(10)

{'预订': 9, '出发': 1, '设计': 8, '性能': 4, '到达': 2, '行程': 6, '机上': 5, '计划': 7, '中转': 0, '售后': 3}


,review_tokens,label_encoded,predicted_labels,predicted_class
0,机票 报销 凭证 没收\n,NaN,9.0,预订
1,南航 月 日 航班 取消 成行 取消 订单\n,NaN,1.0,出发
2,服务 位置 舒服\n,NaN,5.0,机上
3,原坐 航班号 日 时间 起飞 上海 沈阳 更换 时间 三次 最终 延误 日 起飞 太 不靠 ...,NaN,3.0,售后
4,明珠 卡 密码 记不住\n,NaN,3.0,售后
5,月 号 买 南航 航班 上海 虹桥 广州 沒有 里程\n,NaN,4.0,性能
6,我要 退票 打电话 没人接 玩意\n,NaN,6.0,行程
7,出行 原因 改为 泰国 菲律宾 票 退 显示 审核 打款\n,NaN,3.0,售后
8,全程 小时 边上 两个 人用 全 飞机 听 声音 讲 小时 全程 空乘 人员 提醒 真的 很...,NaN,3.0,售后
9,超级 慢 页面 显示\n,NaN,0.0,中转


In [ ]:
# TODO: match original reviews to predicted labels